CreateDatasets
=====
***

## Importing modules
This jupyter notebook creates labelled data from the raw acceleration magnitude streams. It does not explain the format of the UniMiB-SHAR dataset that was primarily used (for that, see InvestigateDataset.ipynb).

The first step is to import the modules needed for calculation and data processing.
* `numpy` is necessary for various statistic calculations used in obtaining features from signal windows such as the mean (average), standard deviation, IQR (inter-quartile range), calculating the sum of signals (discrete integral), and performing Fourier Transforms when calculating power and energy features.
* `scipy.signal` is necessary for estimating auto-spectral density of signals (`welch`) and coherence (`coherence`) of features
* `scipy.io` is used for loading the UniMiB-SHAR dataset from a MatLab (`.mat`) file into numpy arrays
* `math.modf` is a function that splits a floating point number into its whole and decimal parts. It is used for splitting the signals into windows. `math.sqrt` takes the square root of a number and is used for `nperseg` calculations.
* `itertools.combinations` is a function that returns all possible combinations (nCr) from a list of length *n* and specified *r*

In [21]:
import numpy as np
from scipy.signal import welch, coherence
import scipy.io as sio
from math import modf, sqrt
from itertools import combinations

from pprint import pprint

## Feature Extraction
Next is deciding what features we wish to extract from each window of the accelerometer streams. We chose the same features that were used in [similar research](https://www.sciencedirect.com/science/article/pii/S1574119212000703 "Recognizing whether sensors are on the same body"): mean, standard deviation, variance, mean absolute deviation, IQR, energy, and power.

In [22]:
def get_mean(sig):
    '''Returns the mean value of a signal'''
    return np.mean(sig)

def get_std(sig):
    '''Returns the standard deviation of a signal'''
    return np.std(sig)

def get_variance(sig):
    '''Returns the variance of a signal'''
    return get_std(sig)**2

def get_mad(sig):
    '''Returns the mean absolute deviation of a signal'''
    m = get_mean(sig)
    sig = [ abs(x - m) for x in sig ]
    return get_mean(sig)

def get_iqr(sig):
    '''Returns the IQR of a signal'''
    q75, q25 = np.percentile(sig, [75 ,25])
    return q75 - q25

def get_energy(sig, dt):
    '''
    Returns the energy of a signal, approximated using scipy.signal.welch
        sig - the signal
        dt - the time difference between successive samples in sig
    '''
    f_welch, S_xx_welch = welch(sig, fs=1/dt, nperseg=int(sqrt(len(sig))))
    df_welch = f_welch[1] - f_welch[0]
    f_fft = np.fft.fftfreq(len(sig), d=dt)
    df_fft = f_fft[1] - f_fft[0]
    E_welch = (1. / dt) * (df_welch / df_fft) * np.sum(S_xx_welch)
    return E_welch

def get_power(sig, dt):
    '''
    Returns the power of a signal, approximated using scipy.signal.welch
        sig - the signal
        dt - the time difference between successive samples in sig
    '''
    f_welch, S_xx_welch = welch(sig, fs=1/dt, nperseg=int(sqrt(len(sig))))
    df_welch = f_welch[1] - f_welch[0]
    P_welch = np.sum(S_xx_welch) * df_welch
    return P_welch

Then we write a function to return a vector of these features, given a window (section of an accelerometer stream). In `get_feature_matrix`, the signal is split into windows of equal length *w*. Then each of these windows is passed to `get_feature_vector` which returns the corresponding feature vector. That vector is then added to the feature matrix, which is returned from `get_feature_matrix` after all windows have been processed.

In [23]:
def get_feature_vector(window, dt):
    '''
    Returns the feature vector of a window
        window - the window used to calculate the feature vector
        dt - the time difference between successive samples in sig
    Features are: mean, std, variance, mad, iqr, energy, and power
    '''
    mean     = get_mean(window)
    std      = get_std(window)
    variance = get_variance(window)
    mad      = get_mad(window)
    iqr      = get_iqr(window)
    energy   = get_energy(window, dt)
    power    = get_power(window, dt)
    return np.array([mean, std, variance, mad, iqr, energy, power])

def get_feature_matrix(sig, w, dt):
    '''
    Returns the feature matrix of a signal
        sig - the signal used
        w - the length of each window for extracting feature vectors
        dt - the time difference between successive samples in sig
    '''
    windows = split(sig, w)
        
    matrix = []
    for window in windows:
        f = get_feature_vector(window, dt)
        matrix.append(f)
    
    return np.array(matrix)

And last a few helper functions to split a signal into windows and get the *c* value used when calculating coherence...

In [24]:
def split(sig, w):
    '''
    Splits a signal into windows. Returns an array of smaller signals of length w
        sig - the signal to split
        w - the window length in samples
    '''
    num_windows = float(len(sig)) / w
    dec, i = modf(num_windows)
    if num_windows != int(num_windows):
        cutoff = dec * w
        last = int(round(-1*cutoff))
        sig = sig[:last]
        num_windows = i
    return np.split(sig, num_windows)

def get_c(coherence_window, w, dt):
    '''
    Converts coherence window (seconds) into c (windows). Used when determining number of samples at a time 
    for coherence calculation.
    '''
    return int((coherence_window) / (w * dt))

## Coherence

Next we write functions for calculating the normalized coherence of features across a window *c*, and then the normalized coherence of the whole matrix. Each cell of the normalized coherence matrix corresponds to calculating the normalized coherence starting in the same row/column, and including features from *c* rows down. This normalized coherence matrix essentially represents how well two signals are correlated with respect to each feature.


# ( !!!!! TODO : ADD A VISUALIZATION OF ^ !!!!! )

In [25]:
def feature_coherence(f1, f2, phi_max, dt):
    '''
    Calculate the normalized coherence of two feature observances f1 and f2 with a specified phi_max
        f1 - one of the feature observances
        f2 - the other feature observance
        phi_max - the maximum frequency to be considered
        dt - the time difference between successive samples in sig
    '''
    f, C_xy = coherence(f1, f2, 1/(dt * w), nperseg=int(sqrt( (len(f1) + len(f2)) / 2 )) )
    f[f < phi_max]
    C_xy = C_xy[:len(f)]
    return 1/float(phi_max) * np.sum(C_xy)

def matrix_coherence(A, B, c, phi_max, dt):
    '''
    Calculate the normalized coherence matrix of two feature matrices A and B
        A - a feature matrix
        B - a feature matrix
        c - the number of windows considered at a time when calculating coherence of feature observances
        phi_max - the maximum frequency considered
        dt - the time difference between successive samples in sig
    '''
    num_windows = len(A)
    rows = num_windows - (c - 1)
    num_features = len(A[0])
    matrix = np.empty([rows, num_features])

    for f in range(0, 7):
        A_feature = np.transpose(A)[f]
        B_feature = np.transpose(B)[f]

        for k in range(0, rows):
            A_samples = A_feature[k:k+c]
            B_samples = B_feature[k:k+c]
            cell = feature_coherence(A_samples, B_samples, phi_max, dt)

            matrix[k][f] = cell

    return matrix

Now that we have all functions necessary to compute what we're after, the normalized coherence matrix given two magnitude streams, we must write functions to generate the labelled data used in the machine learning algorithms.

***

## Reorganizing the raw data

First, two functions `get_all_trials_UniMiB-SHAR`, and `get_all_trials_COLLECTED` to loop through the raw data (UniMiB-SHAR data and data we collected manually, respectively), and reorganize it into a list of every magnitude stream along with the person it came from. Then `get_pairs` that loops through the list of all trials and returns a list of tuples, each containing a pair of accelerometer streams; and the whole list containing every possible combination.

In [26]:
def get_all_trials_UniMiBSHAR(data, people, activities):
    '''
    Loops through the raw UniMiB-SHAR dataset and reorganizes all requested data into an easier to use list
        data - the raw dataset consisting of many nested numpy arrays
        activities - the requested activities
    '''
    all_trials = []
    # Loop through dataset and append trials to new_data
    for p in people:
        accel_data = data[p][0][0][0]
        
        # Loop through trials and append magnitude streams to trial list
        for a in activities:
            activity = accel_data[a]
            for t in range(len(activity)):
                trial = activity[t][0]
                magnitude = trial[5]
                all_trials.append([p, magnitude])
                
    return all_trials

def get_all_trials_OURS(data):
    '''
    Loops through our raw dataset and reorganizes all requested data into an easier to use list
        data - the raw dataset consisting of many nested numpy arrays
        activities - the requested activities
    '''
    all_trials = []
        
    for p, person in enumerate(data):
        for trial in person:
            for side in trial:
                magnitude = side[1]
                all_trials.append([p, np.array(magnitude)])
                
    # have to delete 15 and 22 because it is very short (sampling must have messed up)
    del all_trials[15]
    del all_trials[22]
    
    return all_trials

def get_pairs(trials):
    '''
    Get a list of all possible combinations of magnitude streams from a list of all trials
    '''
    return list(combinations(trials, 2))

## Processing the data

Once we have a list of all possible combinations of trials and the people they belong to, we can calculate a coherence matrix in `get_coherence_matrix` using the functions we defined above from each pair. This matrix is then labelled in `process` with a `1` if the two streams came from the same person, or a `0` if they came from different people.

In [27]:
def get_coherence_matrix(trial1, trial2, w, c, dt):
    '''
    Calculate a coherence matrix of two magnitude streams and label it
        trial1 - The first trial of the format [person index, magnitude stream]
        trial2 - The second trial of the format [person index, magnitude stream]
        w - the window length used when extracting feature vectors
        c - the coherence window used when calculating feature coherences
        dt - the time difference between successive samples in sig
    '''
    person1 = trial1[0]
    sig1 = trial1[1]
    person2 = trial2[0]
    sig2 = trial2[1]
    
    short = min(len(sig1), len(sig2))
    sig1 = sig1[:short]
    sig2 = sig2[:short]
    
    A = get_feature_matrix(sig1, w, dt)
    B = get_feature_matrix(sig2, w, dt)
    phi_max = 10
    return matrix_coherence(A, B, c, phi_max, dt)

def process(pairs, w, c, dt, verbose=False):
    '''
    Loops through all pairs of trials, calculates the coherence matrix, labels it appropriately, then splits
    each by its rows and labels those with the corresponding label.
        pairs - the list of all pairs of trials
        w - the window length used when extracting feature vectors
        c - the coherence window used when calculating feature coherences
        dt - the time difference between successive samples in sig
    '''
    labelled_matrices = []
    for pair in pairs:
        if verbose >= 2:
            print('{} {}'.format(str(pair[0][0]), str(pair[1][0])))
        labelled_matrices.append([get_coherence_matrix(*pair, w, c, dt), (pair[0][0] == pair[1][0])])
    
    labelled_rows = []
    for x in labelled_matrices:
        new_row = []
        matrix = x[0]
        label = x[1]
        for fc in matrix:
            labelled_rows.append([list(fc), label])
    
    return labelled_rows

Next, we write a fucntion to create a dataset given a list of people, a list of activities, window length, and coherence length. If person 18 is included, we remove them since the data recorded for them in the UniMiB-SHAR dataset were much shorter than the rest and thus couldn't be used effectively. To generate the labelled dataset we simply use `get_all_trials_UniMiBSHAR` and `get_pairs` in succession to get all possible pairs of accelerometer streams. Then we use `get_c` to convert a coherence window in seconds to windows and finally `process` to get the labelled data we have been after this whole time.

(Because of how the data is processed, some combinations of *w* and *cw* don't work together. When this happens, we want to skip the generating the file)

In [28]:
def create_dataset_UniMiBSHAR(full_data, people, activities, w, cw, dt, verbose=1):
    '''
    Creates a labelled dataset from the UniMiB-SHAR raw data given people, activities, w, cw, and dt
        people - a list of people to be included in the dataset
        activities - a list of activities to be included in the dataset
        w - the window length used when extracting feature vectors
        c - the coherence window used when calculating feature coherences
        dt - the time difference between successive samples in sig
    '''
    if 19 in people:
        people.remove(19) # delete person index 19 because short stream
    
    if verbose >= 1:
        print('creating dataset with\n  people {}\n  activities {}\n  w={}\n  cw={}'.format(people, activities, w, cw))
    
    try:
        trials = gete_all_trials_UniMiBSHAR(full_data)
        c = get_c(cw, w, dt)
        labelled = process(pairs, w, c, dt, verbose)
        return labelled
        
    except Exception as e:
        print('Invalid combination of w and cw, file skipped')
        print('(error: {})'.format(e))
        return

    
    
def create_dataset_OURS(data, w, cw, dt, verbose=1):
    '''
    Creates a labelled dataset given from our manually collected raw data
        people - a list of people to be included in the dataset
        activities - a list of activities to be included in the dataset
        w - the window length used when extracting feature vectors
        c - the coherence window used when calculating feature coherences
        dt - the time difference between successive samples in sig
    '''
    trials = get_all_trials_OURS(data)
    pairs = get_pairs(trials)
    c = get_c(cw, w, dt)
    labelled = process(pairs, w, c, dt, verbose)
    return labelled

Now we finally have everything that we need to generate a dataset. First we load the raw `.mat` file into a variable using `scipy.io` and get the actual data into the variable `full_data`. Next we define `dt` depending on the sampling rate of the dataset being used (UniMiB-SHAR is 50 Hz). Last but not least, we call the `create_dataset` function and store what it returns in a variable. This can now be used with classification machine learning algorithms!

In [29]:
# mat = sio.loadmat('./data/UniMiB-SHAR/data/full_data.mat') # UniMiB-SHAR dataset
# full_data = mat['full_data']

data = np.load('./data/collected_clean/magnitude_data_clean.npy')

dt = 0.02

# people = list(range(30))
# activities = [2]
w = 4
cw = 15

# dataset = create_dataset_UniMiBSHAR(data, people, activities, w, cw, dt, verbose=2)
dataset = create_dataset_OURS(data, w, cw, dt, verbose=2)

0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 1
0 1
0 1
0 1
0 1
0 1
0 1
0 1
0 1
0 1
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 1
0 1
0 1
0 1
0 1
0 1
0 1
0 1
0 1
0 1
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 1
0 1
0 1
0 1
0 1
0 1
0 1
0 1
0 1
0 1
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 1
0 1
0 1
0 1
0 1
0 1
0 1
0 1
0 1
0 1
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 1
0 1
0 1
0 1
0 1
0 1
0 1
0 1
0 1
0 1
0 0
0 0
0 0
0 0
0 0
0 0
0 1
0 1
0 1
0 1
0 1
0 1
0 1
0 1
0 1
0 1
0 0
0 0
0 0
0 0
0 0
0 1
0 1
0 1
0 1
0 1
0 1
0 1
0 1
0 1
0 1
0 0
0 0
0 0
0 0
0 1
0 1
0 1
0 1
0 1
0 1
0 1
0 1
0 1
0 1
0 0
0 0
0 0
0 1
0 1
0 1
0 1
0 1
0 1
0 1
0 1
0 1
0 1
0 0
0 0
0 1
0 1
0 1
0 1
0 1
0 1
0 1
0 1
0 1
0 1
0 0
0 1
0 1
0 1
0 1
0 1
0 1
0 1
0 1
0 1
0 1
0 1
0 1
0 1
0 1
0 1
0 1
0 1
0 1
0 1
0 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1


If the dataset was successfully created, we can save it.

In [30]:
# filename = 'p' + str(len(people)) + '_a' + str(activities) + '_w' + str(w) + '_cw' + str(cw) + '.npy'
# np.save('./data/created_UniMiB-SHAR/nperseg=sqrt/' + filename, dataset)

filename = 'w' + str(w) + '_cw' + str(cw) + '.npy'
np.save('./data/created_collected/' + filename, dataset)

Since creating a dataset can take a while, a function that can create multiple datasets automatically would be very useful (for example running it overnight). To do so we will write a function that loops through all possible sets of people, activities, window lengths, and coherence windows specified and create and save a dataset for each with a descriptive name.

In [31]:
def create_datasets(people_sets, activities, window_lengths, coherence_windows, dt, verbose=1):
    '''
    Creates multiple datasets and saves them all to a file
        people_sets - a list of lists of people from which to create datasets
        activities - a list of activities to be included in the dataset
        window_lengths - window lengths to be used
        coherence_windows - coherence windows to be used
        dt - the time difference between successive samples in sig
    '''
    for people in people_sets:
        for w in window_lengths:
            for cw in coherence_windows:
                filename = 'p' + str(len(people)) + '_a' + str(activities) + '_w' + str(w) + '_cw' + str(cw) + '.npy'
                if verbose >= 1:
                    print('starting {}'.format(filename))
                try:
                    dataset = create_dataset(people, activities, w, cw, dt, verbose)
                    np.save('./data/nperseg=sqrt' + filename, dataset)
                except KeyboardInterrupt:
                    raise
                finally:
                    print('finished {}'.format(filename))

Now we can finally move on to the fun part: machine learning.